# Билет 28

## Задание 1

## Задание 2

<img src="img/28_2.png" width=800>

In [5]:
import nltk
import itertools

In [6]:
def read_text(path: str) -> str:
    with open(path, encoding='cp1251') as f:
        return f.read()

In [7]:
def close_sents(sents: list[str]):
    for i, s1 in enumerate(sents):
        for s2 in sents[i + 1:]:
            if abs(len(s1) - len(s2)) <= 3:
                distance = nltk.edit_distance(s1, s2)
                if distance <= 3:
                    yield distance, (s1, s2)

In [8]:
text = read_text('data/task_2/AnnaKarenina_.txt')
sents = list({s for s in nltk.sent_tokenize(text) if len(s) <= 40})
list(itertools.islice(close_sents(sents), 10))

[(2, ('– обратился он к нему.', '– обратился он к ней.')),
 (3, ('– обратился он к нему.', '– обратился он к даме.')),
 (3, ('Ну, все равно.', 'Все, все равно.')),
 (3, ('«Вот оно!', 'Вон он!')),
 (3, ('«Вот оно!', 'Вон они!')),
 (3, ('«Вот оно!', 'вот он!')),
 (3, ('«Вот оно!', '– Вот он!')),
 (2, ('«Вот оно!', 'Вот он!')),
 (1, ('«Вот оно!', 'Вот оно!')),
 (2, ('«Вот оно!', '–\xa0Вот оно!'))]

## Задание 3

<img src="img/28_3.png" width=800>

In [9]:
import numpy as np
import pandas as pd
import multiprocessing as mp
from collections import defaultdict

In [10]:
def smart_count(s: str) -> bool:
    cnt = defaultdict(int)
    for ch in s:
        if cnt[ch] >= 8:
            return True
        cnt[ch] += 1
    return False


def solve(ints_sample) -> list[tuple[int, float]]:
    sin_s = pd.Series(np.sin(ints_sample), index=ints_sample)
    mask = sin_s.apply(lambda s: smart_count(str(s)))
    return list(sin_s[mask].items())

In [11]:
# %%file "src/task_28.py"
# import pandas as pd
# from collections import defaultdict
# import numpy as np
#
#
# def smart_count(s: str) -> bool:
#     cnt = defaultdict(int)
#     for ch in s:
#         if cnt[ch] >= 8:
#             return True
#         cnt[ch] += 1
#     return False
#
#
# def solve(ints_sample) -> list[tuple[int, float]]:
#     sin_s = pd.Series(np.sin(ints_sample), index=ints_sample)
#     mask = sin_s.apply(lambda s: smart_count(str(s)))
#     return list(sin_s[mask].items())

In [12]:
def blocking_mode(ints_sample) -> list[tuple[int, float]]:
    return solve(ints_sample)


def nonblocking_mode(ints_sample) -> list[tuple[int, float]]:
    from src.task_28 import solve

    def chunkinize(lst, n: int):
        n = len(lst) // n
        for i in range(0, len(lst), n):
            yield lst[i:i + n]

    n = mp.cpu_count()
    with mp.Pool(n) as pool:
        res = pool.map(solve, chunkinize(ints_sample, n))
        return [i for chunk in res for i in chunk]

In [13]:
ints_sample = np.arange(1, 400_000 + 1)

In [14]:
blocking_mode(ints_sample)[:10]

[(1797, 0.009002025051800673),
 (3775, -0.9299841096999939),
 (7089, 0.9999926930119999),
 (10639, 0.9999937999413936),
 (24074, 0.024502006300003643),
 (25904, -0.9999975242999319),
 (31249, 0.4097000800060077),
 (36693, -0.7188828881448883),
 (38968, -0.31007805001001504),
 (46617, 0.8881880887188651)]

In [15]:
nonblocking_mode(ints_sample)[:10]

[(1797, 0.009002025051800673),
 (3775, -0.9299841096999939),
 (7089, 0.9999926930119999),
 (10639, 0.9999937999413936),
 (24074, 0.024502006300003643),
 (25904, -0.9999975242999319),
 (31249, 0.4097000800060077),
 (36693, -0.7188828881448883),
 (38968, -0.31007805001001504),
 (46617, 0.8881880887188651)]

In [16]:
%%timeit

blocking_mode(ints_sample)

1.65 s ± 56 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%%timeit

nonblocking_mode(ints_sample)

1.37 s ± 24.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
